# Optuna study

> Combine it with papermill and wandb for seamless hyperparameter tuning

In [1]:
import optuna
from tsai.optuna import *
import papermill as pm
from tsai.optuna import run_optuna_study
from fastcore.basics import *

In [ ]:
config = AttrDict(
    study_name = 'general',
    n_trials = 20,
    train_nb = '/home/gridsan/vrodriguez/Github/sw-driver-forecaster/dev_nbs/training.ipynb' # absolute path
)

In [ ]:
def objective(trial:optuna.Trial, train_nb, wandb_group=None):
    # Define search space here
    attn_dropout = trial.suggest_float('attn_dropout', 0.0, 0.5, step=.1)
    dropout = trial.suggest_float('dropout', 0.0, 0.5, step=.1)
    d_model = trial.suggest_int('d_model', 64, 512, 64)
    n_layers = trial.suggest_int('n_layers', 2, 5, 1)
    d_ff = trial.suggest_int('d_ff', 64, 512, 64)
    lr_max = trial.suggest_float("lr_max", 1e-5, 1e-2, log=True)

    # Call the training notebook using papermill (don't print the output)
    stdout_file = open('tmp/pm_stdout.txt', 'w')
    stderr_file = open('tmp/pm_stderr.txt', 'w')
    pm.execute_notebook(
        train_nb,
        './tmp/pm_output.ipynb',
        parameters = {
            'config.attn_dropout': attn_dropout,
            'config.dropout': dropout,
            'config.d_model': d_model,
            'config.d_ff': d_ff,
            'config.n_layers': n_layers,
            'config.lr_max': lr_max,
            'config.
        },
        stdout_file = stdout_file,
        stderr_file = stderr_file
    )

    # Close the output files
    stdout_file.close()
    stderr_file.close()

    # Get the output value of interest from the source notebook
    %store -r valid_loss
    return valid_loss       

In [ ]:
run_optuna_study(objective, study_type='bayesian', direction='minimize', path='./tmp',
                 study_name=config.study, n_trials=config.n_trials)